In [1]:
complete_dataset_url="http://files.grouplens.org/datasets/movielens/ml-latest.zip"
small_dataset_url="http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

In [20]:
import os
datasets_path=os.path.join('file://home/hduser/','datasets')
complete_dataset_path=os.path.join(datasets_path,'ml-latest.zip')
small_dataset_path=os.path.join(datasets_path, 'ml-latest-small.zip')


In [21]:
import urllib


In [22]:
small_f=urllib.urlretrieve(small_dataset_url,small_dataset_path)
complete_f=urllib.urlretrieve(complete_dataset_url,complete_dataset_path)

IOError: [Errno 2] No such file or directory: 'file://home/hduser/datasets/ml-latest-small.zip'

In [10]:
import zipfile

with zipfile.ZipFile(small_dataset_path,"r") as z:
    z.extractall(datasets_path)
with zipfile.ZipFile(complete_dataset_path,"r") as z:
    z.extractall(datasets_path)

In [1]:
#small_ratings_file=os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

#from scala.io import Source
#small_ratings_data=scala.io.Source.fromFile(small_ratings_file)
small_ratings_raw_data=sc.textFile("/user/hduser/ml-latest-small/ratings.csv")


In [31]:
print small_ratings_raw_data.count

<bound method RDD.count of /user/hduser/ml-latest-small/ratings.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0>


In [2]:
small_ratings_raw_data_header=small_ratings_raw_data.take(1)[0]

In [3]:
small_ratings_data=small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [4]:
small_ratings_data.count

[(u'1', u'31', u'2.5'), (u'1', u'1029', u'3.0'), (u'1', u'1061', u'3.0')]

In [5]:
small_movies_raw_data=sc.textFile("/user/hduser/ml-latest-small/movies.csv")

In [6]:
small_movies_raw_data_header=small_movies_raw_data.take(1)[0]
small_movies_data=small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens:(tokens[0], tokens[1])).cache()

In [38]:
small_movies_data.take(3)

[(u'1', u'Toy Story (1995)'),
 (u'2', u'Jumanji (1995)'),
 (u'3', u'Grumpier Old Men (1995)')]

In [7]:
training_RDD, validation_RDD, test_RDD=small_ratings_data.randomSplit([6,2,2], seed=0L)
validation_for_predict_RDD=validation_RDD.map(lambda x:(x[0], x[1]))
validation_RDD.take(1)[0]

(u'1', u'1129', u'2.0')

In [8]:
test_for_predict_RDD=test_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD.take(1)[0]

(u'1', u'31')

In [9]:
from pyspark.mllib.recommendation import ALS
import math

In [13]:
seed=5L
iterations=10
regularization_parameter=0.1
ranks=[4,8,12]
errors=[0,0,0]
err=0
tolerance=0.02

min_error=float('inf')
best_rank=-1
best_iteration=-1
i=0
while i<1:
    i+=1
    model=ALS.train(training_RDD,8,seed=seed, iterations=5, lambda_=regularization_parameter)
    predictions=model.predictAll(validation_for_predict_RDD).map(lambda r:((r[0],r[1]),r[2]))
    rates_and_preds=validation_RDD.map(lambda r:((int(r[0]),int(r[1])),float(r[2]))).join(predictions)
    error=math.sqrt(rates_and_preds.map(lambda r: ((r[1][0]-r[1][1])**2)).mean())
    errors[err]=error
    err+=1
    print 'for rank the error is %s'%error
#    if error<min_error:
#        min_error=error
#        best_rank=rank
#    print 'Best model is with rank %s'%best_rank

for rank the error is 0.958315656133


In [14]:
predictions.take(3)

[((529, 1084), 3.593032230014175),
 ((605, 1084), 3.5634433212055594),
 ((547, 1084), 3.4348652176683623)]

In [16]:
rates_and_preds.take(5)

[((198, 4772), (2.5, 3.6396592564244195)),
 ((472, 2014), (3.0, 3.182789270177393)),
 ((346, 2000), (5.0, 3.4026644981358)),
 ((146, 736), (3.5, 3.025318378517568)),
 ((63, 2987), (4.5, 3.6139026390520055))]

In [11]:
model=ALS.train(training_RDD, 8, seed=seed, iterations=5, lambda_=regularization_parameter)
predictions=model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0],r[1]),r[2]))
rates_and_preds=test_RDD.map(lambda r: ((int(r[0]),int(r[1])),float(r[2]))).join(predictions)
error=math.sqrt(rates_and_preds.map(lambda r: (r[1][0]-r[1][1])**2).mean())
print 'error %s'%error

NameError: name 'seed' is not defined

In [1]:
complete_ratings_raw_data=sc.textFile("/user/hduser/ml-latest/ratings.csv")

In [2]:
complete_ratings_header=complete_ratings_raw_data.take(1)[0]

In [3]:
complete_ratings_header

u'userId,movieId,rating,timestamp'

In [17]:
complete_ratings_data=complete_ratings_raw_data.filter(lambda line: line!= complete_ratings_header).map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()

In [7]:
complete_ratings_data.count()

26024289

In [18]:
from pyspark.mllib.recommendation import ALS
import math
training_RDD,test_RDD=complete_ratings_data.randomSplit([7,3],seed=0L)
complete_model=ALS.train(training_RDD, 8, seed=5L,iterations=5, lambda_=0.1)
test_for_predict_RDD=test_RDD.map(lambda x: (x[0],x[1]))
predictions=complete_model.predictAll(test_for_predict_RDD).map(lambda x: ((x[0],x[1]),x[2]))
#rates_and_preds=test_for_predict_RDD.map(lambda x: ((int(x[0]),int(x[1])),float(x[2]))).join(predictions)
#error=math.sqrt(rates_and_preds.map(lambda x: (x[1][0]-x[1][1])**2).mean())
#print 'error is %s'%error

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 128.0 failed 4 times, most recent failure: Lost task 0.3 in stage 128.0 (TID 133, abc1-VirtualBox, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/spark/python/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-18-2132d05009c7>", line 7, in <lambda>
IndexError: tuple index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at or

In [21]:
predictions.take(3)

[((266991, 26094), 3.8813126385906083),
 ((266991, 3408), 3.6700687813557265),
 ((266991, 104841), 3.5693392849545664)]

In [22]:
rates_and_preds=test_RDD.map(lambda x: ((int(x[0]),int(x[1])),float(x[2]))).join(predictions)

In [23]:
rates_and_preds.take(3)

[((192898, 848), (2.0, 3.013263845108634)),
 ((2821, 736), (3.0, 3.710995883099808)),
 ((93345, 3578), (4.5, 4.255822859665534))]

In [25]:
#error=math.sqrt(rates_and_preds.map(lambda x: (x[1][0]-x[1][1])**2).mean())
#print 'error %s'%error

#save the model
#from pyspark.mllib.recommendation import MatrixFactorizationModel
#complete_model.save(sc, "/user/hduser/complete_model_als")
#same_model=MatrixFactorization.load(sc,"/user/hduser/complete_model_als")

#load movies dataset
complete_movies_raw_data=sc.textFile("/user/hduser/ml-latest/movies.csv")
complete_movies_header=complete_movies_raw_data.take(1)[0]
complete_movies_data=complete_movies_raw_data.filter(lambda line: line!=complete_movies_header).map(lambda line: line.split(","))

In [26]:
complete_movies_data.take(1)

[[u'1', u'Toy Story (1995)', u'Adventure|Animation|Children|Comedy|Fantasy']]

In [27]:
complete_movies_data=complete_movies_data.map(lambda x: (int(x[0]),x[1],x[2])).cache()
complete_movies_titles=complete_movies_data.map(lambda x:(int(x[0]),x[1]))


In [55]:
def get_counts_and_averages(ID_and_ratings_tuple):
    n_ratings=len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0],(n_ratings, float(sum(x for x in ID_and_ratings_tuple[1]))/n_ratings)

movie_ID_with_ratings_RDD=complete_ratings_data.map(lambda x: (x[1],x[2])).groupByKey()
movie_ID_with_avg_ratings_RDD=movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_ratings_counts_RDD=movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0],x[1][0]))


In [31]:
new_user_ID=0
new_user_ratings=[(0,260,4), (0,1,3), (0,16,3), (0,25,4), (0,32,4), (0,335,1), (0,379,1), (0,296,3), (0,858,5), (0,50,4)]
new_user_ratings_RDD=sc.parallelize(new_user_ratings)


In [32]:
complete_data_with_new_ratings_RDD=complete_ratings_data.union(new_user_ratings_RDD)

In [33]:
from time import time
t0=time()
new_ratings_model=ALS.train(complete_data_with_new_ratings_RDD, 8, seed=5L, iterations=5, lambda_=0.1)
tt=time()-t0
print 'new model in %s seconds'%round(tt,3)


new model in 562.241 seconds


In [47]:
new_users_ratings_ids=map(lambda x: x[1], new_user_ratings)
new_user_unrated_movies_RDD=complete_movies_data.filter(lambda x: x[0] not in new_users_ratings_ids).map(lambda x: (new_user_ID, x[0]))



In [48]:
new_user_recommendations_RDD=new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [50]:
new_user_recommendations_RDD.take(3)

[Rating(user=0, product=152288, rating=1.9851817436807833),
 Rating(user=0, product=32196, rating=2.822296060777252),
 Rating(user=0, product=143464, rating=1.0014514313432192)]

In [56]:
new_user_recommendations_rating_RDD=new_user_recommendations_RDD.map(lambda x:(x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD=new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_ratings_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(98304, ((1.4132888625369526, u'So Big! (1932)'), 4)),
 (147456, ((-0.4480970023792108, u'The Legend of Longwood (2014)'), 2)),
 (131076, ((1.675362413861187, u'S\xfcperseks (2004)'), 3))]

In [57]:
new_user_recommendations_rating_title_count_RDD=new_user_recommendations_rating_title_and_count_RDD.map(lambda r:(r[1][0][1], r[1][0][0],r[1][1]))
top_movies=new_user_recommendations_rating_title_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])
print 'top recommended movies %s'%'\n'.join(map(str,top_movies))



top recommended movies (u'"Godfather: Part II', 3.973247956789318, 36679)
(u'Death on the Staircase (Soup\xe7ons) (2004)', 3.8345034719015834, 95)
(u'"Enemies of Reason', 3.833799907146231, 29)
(u'Death Note: R2 - L o Tsugu Mono (2008)', 3.759341310119992, 25)
(u"One Flew Over the Cuckoo's Nest (1975)", 3.738813257703223, 40103)
(u'Goodfellas (1990)', 3.7357950741250923, 33987)
(u'Apocalypse Now (1979)', 3.716516869419345, 27741)
(u'Pinchcliffe Grand Prix (Fl\xe5klypa Grand Prix) (1975)', 3.7156292551933676, 33)
(u'Raging Bull (1980)', 3.713228335505476, 11365)
(u'"Shawshank Redemption', 3.7029253086069596, 91082)
(u"Long Night's Journey Into Day (2000)", 3.697104935526685, 35)
(u'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 3.6936398512966573, 28280)
(u'The Price of Gold (2014)', 3.691081114384569, 25)
(u"Schindler's List (1993)", 3.6834406676495663, 67662)
(u'Band of Brothers (2001)', 3.6720794497922684, 284)
(u'Fargo (1996)', 3.671669162377852, 52474

In [61]:
my_movie=sc.parallelize([(0,500)])
individual_movie_rating_RDD=new_ratings_model.predictAll(my_movie)
individual_movie_rating_RDD.count()

1

In [62]:
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=500, rating=2.2813049313853764)]